In [1]:
!pip install openpyxl
!pip install selenium

import pandas as pd
import openpyxl
import time 
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
file_path = "extracted_data_clothing.xlsx"
df = pd.read_excel(file_path)
companies = df['Company Name']
headers = df.columns.tolist()
column_index = 0
for index, header in enumerate(headers):
    if header.strip() == 'Result':
        column_index = index + 1

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("log-level=3")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--start-miniimized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

In [4]:
driver.get('https://sasb.org/standards/materiality-finder/find/')
search_box = driver.find_element(By.XPATH, '//input[@data-testid="company-search-form-input"]')
search_box.clear()

In [ ]:
words_to_remove = ['.', 's.p.a.']

def update_excel_cell(row_index, new_data):
    try:
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active
        cell = sheet.cell(row=row_index, column=column_index)
        cell.value = new_data

        workbook.save(file_path)
    except Exception as e:
        print(f"An error occurred: {e}")
        
def remove_right_portion(sentence, words_to_remove):
    for word in words_to_remove:
        idx = sentence.find(word)
        if idx != -1:
            sentence = sentence[:idx]
            break
    return sentence

for row, name in enumerate(companies):
    filtered_name = remove_right_portion(name.lower(), words_to_remove)
    search_box.clear()
    search_box.send_keys(filtered_name.strip())
    driver.find_element(By.XPATH, '//button[@data-testid="company-search-form-submit"]').click()
    time.sleep(3)
    result = driver.find_element(By.XPATH, '//div[@role="status"]').text
    if 'Found' in result:
        update_excel_cell(row+2, 'Found')
        print(f'{row+1} -> {name} -> Found')
    else:
        update_excel_cell(row+2, 'Not Found')
        print(f'{row+1} -> {name} -> Not Found')
    
        

    

1 -> A.K. Spintex Ltd. -> Found
2 -> Aananda Lakshmi Spinning Mills Ltd. -> Found
3 -> Aarnav Fashions Ltd. -> Not Found
4 -> Aarvee Denims & Exports Ltd. -> Found
5 -> ABC-Mart Inc. -> Found
6 -> Acelon Chemicals & Fiber Corp. -> Found
7 -> Acknit Industries Ltd. -> Found
8 -> Active Clothing Co. Ltd. -> Found
9 -> Addi Industries Ltd. -> Found
10 -> adidas AG -> Found
11 -> adidas AG -> Found
